Importing Various libraries needed.
Floium for displaying map
geopy for getting location data for a specific location
ssl, certifi to ignore ssl certificate Verification 

Problem Statement
Food is an integral part of the culture of a place. When we visit a place it might be difficult to select a restaurant as we expect different things like ambiance, budget etc. This project uses FourSquare Api to collect restaurants near CP ,Delhi, India. Is used Folium to plot on a map, geolocator to get latitudes and longitude of a neighbourhood. It uses sklearn to make a model which takes reviews and likes of a place and can recommend a place based on your input of string.

In [165]:
import requests
import pandas as pd
import folium
import numpy as np
import geopy.geocoders
from geopy.geocoders import Nominatim
import certifi
import time
import ssl
import csv

I am collecting data of a particular location in Delhi, India. Using address as Connaught Place, Delhi, i will get the location of Connaught Place, Delhi when passed through geolocator.
SSL verifivation is necessary to ignore as it is giving an error on my machine.

In [166]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
geopy.geocoders.options.default_ssl_context = ctx
geolocator = Nominatim()
address = 'Connaught Place, Delhi'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


GetNearbyVenues takes a list entry of name, latitudes, longitudes and default parameter of 500 m
I have declared Client_ID, Client_Secret and version in the beginning.I have a list venue_list to store all the venues nearby the Connaught Place, Delhi. Condition used in api used is selection : food to get food places address, latitude, longitude, name, category and id.
Limit is set to 100 to get max 100 places near the location.

In [167]:
def getNearbyVenues(name, latitudes, longitudes, radius = 500):
    #Pass in list of names, latitude, longitude
    CLIENT_ID = 'FHAQR0ZU3PXJODUWXILUN4H0NAVXMXV0PMVRLA04Q0ILOJPV' # your Foursquare ID
    CLIENT_SECRET = 'M5VK0GYVWI4MXF1SZBTPQWYEBQ2IVZ1Q5FC5AA5EIT3UCEAH' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    venues_list=[]
    print(name)
    limit = 100
    radius = 500
    temp_latitude = latitudes
    temp_longitude = longitudes
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        temp_latitude, 
        temp_longitude, 
        radius, 
        limit)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    venues_list.append([(
        name, 
        latitude, 
        longitude, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name'],
        v['venue']['id']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
            'Neighbourhood Latitude', 
            'Neighbourhood Longitude', 
            'Venue',
            'Venue Latitude', 
            'Venue Longitude', 
            'Venue Category',
            'Venue Id']
    return(nearby_venues)

GetLikes function takes input of names list and their venue id.
Four Square API is called to get the likes of the venue whose venue id is passed.
This information is stored in like_list which is the passed.

In [168]:
import json
def getlikes(names,venue_id, radius = 500):
    CLIENT_ID = 'FHAQR0ZU3PXJODUWXILUN4H0NAVXMXV0PMVRLA04Q0ILOJPV' # your Foursquare ID             
    CLIENT_SECRET = 'M5VK0GYVWI4MXF1SZBTPQWYEBQ2IVZ1Q5FC5AA5EIT3UCEAH' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    like_list=[]
    for name,venid in zip(names, venue_id):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(
        venid,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION)
        like_result = requests.get(url).json()['response']
        like_list.append(like_result['likes']['count'])
    return like_list

Finally calling the function getnearbyvenues on "Connaught Place, Delhi and storing the result in dataframe df

In [169]:
df = getNearbyVenues("Connaught Place",latitude,longitude )

Connaught Place


THis function takes list of venue id and extracts the tip about the venue along with user id. Since calls are limited I am using two accounts for fetching the data. When calls excede 45, it will switch to different ID

In [170]:
def getReviews(venueId):
    count = 0;
    review = []
    userid = []
    for(venId) in zip(venueId):
        if(count < 45):
            CLIENT_ID = 'FHAQR0ZU3PXJODUWXILUN4H0NAVXMXV0PMVRLA04Q0ILOJPV' # your Foursquare ID
            CLIENT_SECRET = 'M5VK0GYVWI4MXF1SZBTPQWYEBQ2IVZ1Q5FC5AA5EIT3UCEAH' # your Foursquare Secret
            VERSION = '20180605' # Foursquare API version
            count  = count+1
        else:
            CLIENT_ID = 'AQJZAR2CDWN5F5SHA1KTVU1U3G1OLXYJVQ3G14H5MZG2Q5OA' # your Foursquare ID
            CLIENT_SECRET = 'Y5RWWZ1UNAQZW01HYWP00KPIUKPDFDNVCI42IWMP1Q0SD4A4' # your Foursquare Secret
            VERSION = '20180605' # Foursquare API version
        url = 'https://api.foursquare.com/v2/venues/{}/tips?&client_id={}&client_secret={}&v={}'.format(
            venId[0],
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION )
        results = requests.get(url).json()
        try:
            review.append(results["response"]['tips']['items'][0]['text'])
            userid.append(results["response"]['tips']['items'][0]['id'])
        
        except:
            review.append("Bad")
            userid.append('1a21ab1c498e411929349f40')
        
    return review,userid

Finally storing the data from getReviews into the dataframe with col names 'Reviews', 'User Id'

In [171]:
df['Reviews'],df['User Id'] = getReviews(df['Venue Id'])

Finally storing the data of likes in df with col name 'LIKES'

In [172]:
df['Likes'] = getlikes(df['Venue'], df['Venue Id'])

Wenger's
Farzi Cafe
Chaayos
Rajdhani Thali
Johnny Rockets
United Coffee House
Nando's
Khan Chacha | खान चाचा | خان چاچا
Pind Balluchi
HOTEL SARAVANA BHAVAN
Tamasha
Barbeque Nation
Berco's
Nizam's Kathi Kabab | निजा़म काठी कबाब
Odeon Social
Immigrants Project - A Cafe in History
Sagar Ratna
Warehouse Cafe
Wenger's Deli
Fresc Co
Dunkin' Donuts
Anand Biryani House
Chew
Shake square
Costa Coffee
Panchayat Paan Parlour
Garam Dharam
Smoke House Deli
Kwality Restaurant
Burger King
Zen Chinese, Connaught Place
McDonald’s
Japanese Fuji Restaurant
Haldiram's
Veda Restaurant
KFC
Domino's Pizza
Ignis
McDonald's
McDonald's
Side Wok
Pizza Hut
Depauls Cold Coffee
Punjabi by Nature Quickie
Kake-Da-Hotel
Comesum
McDonald's
Domino's Pizza
Indian Coffee House
Minar
Fa Yian
Gola Sizzlers
FIRE
Biryani Paradise
MIST
Lady Baga
Caffe9
G
Fifty9
SIDEWOK
The Great Kabab Factory
China Garden
Hira Sweets, CP
The Darzi Bar & Kitchen
Pioneer's Flavour of China
Zizo
Sandoz CP
Flavours Of China


Final data frame check. After this all the collection of data has been completed.

In [173]:
df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id,Reviews,User Id,Likes
0,Connaught Place,28.631383,77.219792,Wenger's,28.633412,77.218292,Bakery,4e4e44902271a1bdc3ccee8d,Bad,1a21ab1c498e411929349f40,173
1,Connaught Place,28.631383,77.219792,Farzi Cafe,28.632581,77.221125,Molecular Gastronomy Restaurant,568feb91498edb833d755933,Bad,1a21ab1c498e411929349f40,25
2,Connaught Place,28.631383,77.219792,Chaayos,28.631629,77.220364,Café,56adb1f8498e9db1d6bb16ed,Bad,1a21ab1c498e411929349f40,13
3,Connaught Place,28.631383,77.219792,Rajdhani Thali,28.629999,77.220401,Indian Restaurant,4cbb2592dd41a35df95eeca0,Bad,1a21ab1c498e411929349f40,25
4,Connaught Place,28.631383,77.219792,Johnny Rockets,28.630457,77.219594,Bistro,571a5505498ef32d7975392f,Bad,1a21ab1c498e411929349f40,8


Writing the data in a csv file so that it can be used later without exhausting all the API calls
Write in made which writes all the entries of df into the file.

In [174]:
with open ("DelhiRestaurants.csv", 'w') as f:
    writer = csv.writer(f, delimiter = ",")
    col = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbour Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category', 'Venue Id', 'Reviews','User Id', 'Likes']
    writer.writerow(col)
    for data in range(0,df.shape[0]):
        writer.writerow(df.iloc[data, :])
df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id,Reviews,User Id,Likes
0,Connaught Place,28.631383,77.219792,Wenger's,28.633412,77.218292,Bakery,4e4e44902271a1bdc3ccee8d,Bad,1a21ab1c498e411929349f40,173
1,Connaught Place,28.631383,77.219792,Farzi Cafe,28.632581,77.221125,Molecular Gastronomy Restaurant,568feb91498edb833d755933,Bad,1a21ab1c498e411929349f40,25
2,Connaught Place,28.631383,77.219792,Chaayos,28.631629,77.220364,Café,56adb1f8498e9db1d6bb16ed,Bad,1a21ab1c498e411929349f40,13
3,Connaught Place,28.631383,77.219792,Rajdhani Thali,28.629999,77.220401,Indian Restaurant,4cbb2592dd41a35df95eeca0,Bad,1a21ab1c498e411929349f40,25
4,Connaught Place,28.631383,77.219792,Johnny Rockets,28.630457,77.219594,Bistro,571a5505498ef32d7975392f,Bad,1a21ab1c498e411929349f40,8


For the final show I am importing sklearn to vectorize the features in reviews and importing the nltk(Natural Languare Tool Kit) to clean the reviews.

In [175]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

This algorithm removes all emoji, punctuation, nouns and other words which are for the language sake and will not provide any valid token.

In [176]:
import string
import re
def clean_text(reviews):
    res = []
    for text in reviews :
        text = text.translate(string.punctuation)

        ## Convert words to lower case and split them
        text = text.lower().split()

        ## Remove stop words
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops and len(w) >= 3]

        text = " ".join(text)

        # Clean the text
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"\-", " - ", text)
        text = re.sub(r"\=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)    
        res.append(text)
    return res

Cleaning the data and rewriting the data in reviews and copying the original comments in teh data frame in Reviews_ori

In [177]:
df['Reviews_ori'] = df['Reviews']
df['Reviews'] = clean_text(df['Reviews'])

Now making separate list frame for user id and venue id

In [178]:
user_df = df[['User Id', 'Reviews']]
venue_df = df[['Venue Id', 'Reviews']]

in user_df I am making a paragraph of all the comments given out by a user and joining them.
This is done with venue_df. All the reviews written about venue_id is made into a paragraph and then added to the corresponding venue_id

In [179]:
user_df = user_df.groupby('User Id').agg({'Reviews':' '.join})
venue_df = venue_df.groupby('Venue Id').agg({'Reviews':' '.join})

Vectorizing all the features of the reviews given out by the user, max features allowed 1000, to make a judgement that what "good" means. Similar process is gone with venues and the check is likes. Is is assumed that restaurant will be consistant with the review and hence words or features can be assigned a meaning.

In [180]:
#User Id vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=1000)
userid_vectors = userid_vectorizer.fit_transform(user_df['Reviews'])
userid_vectors.shape

(15, 128)

In [181]:
#Venue Id vectorizer
venue_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=1000)
venue_vectors = venue_vectorizer.fit_transform(venue_df['Reviews'])
venue_vectors.shape

(68, 128)

Making two data frames with userid, and features for both user and venue and naming that P for user and Q for Venue

In [182]:
P = pd.DataFrame(userid_vectors.toarray(), index=user_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(venue_vectors.toarray(), index=venue_df.index, columns=venue_vectorizer.get_feature_names())


Let us see P

In [183]:
P.head()

,!,15,1940,330ml,650,amazing,ambience,american,approximate,average,...,tried,turbans,unlimited,value,veg,view,waiters,wearing,what,yore
User Id,,,,,,,,,,,,,,,,,,,,,
1a21ab1c498e411929349f40,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
4b7812db70c603bb373b92b4,0.000000,0.0,0.000000,0.0,0.0,0.351237,0.0,0.0,0.000000,0.0,...,0.351237,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
4e8ff234d22dccc37e935193,0.560362,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
4f0b05fee4b071c57819891f,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.368743,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
502a3170e4b022b4b5336955,0.148162,0.0,0.191199,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.191199,0.0,0.0,0.0,0.191199,0.191199,0.191199,0.0,0.191199


Let us see Q

In [184]:
Q.head()

,!,15,1940,330ml,650,amazing,ambience,american,approximate,average,...,tried,turbans,unlimited,value,veg,view,waiters,wearing,what,yore
Venue Id,,,,,,,,,,,,,,,,,,,,,
4b59921bf964a520fe8c28e3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4b5c78e9f964a5206f3129e3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4b7421c5f964a5204ac92de3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4b79606ef964a52089f62ee3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4b9f365df964a520241837e3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Making a matrix with rows as row id and columns as venue id. All the corresponding values whihc are common in both are filles, NaN otherwise. It is expected that a lot of entries will be NaN as not all users have tried each and every restaurant in the city. So not need to freek out.

In [185]:
userid_rating_matrix = pd.pivot_table(df, values='Likes', index=['User Id'], columns=['Venue Id'])
userid_rating_matrix.shape

(15, 68)

In [186]:
userid_rating_matrix

Venue Id,4b59921bf964a520fe8c28e3,4b5c78e9f964a5206f3129e3,4b7421c5f964a5204ac92de3,4b79606ef964a52089f62ee3,4b9f365df964a520241837e3,4badb775f964a520ea643be3,4bf107ab24f020a1e0a8684f,4bfd4ffab68d0f476592e857,4c347c9a213c2d7faf81385d,4c359b0f7cc0c9b6ca33f59a,...,5685691d498eee449e7d6908,568feb91498edb833d755933,569b32b5498ef7c4152b45fb,56adb1f8498e9db1d6bb16ed,56c6e01ccd10173283f8534e,571a5505498ef32d7975392f,5805038038fa153aca5f0a1c,580a6d0738fa3e233b7c0045,5881e608ac13691c906cfa3b,58e9209c469aef21458c8be1
User Id,,,,,,,,,,,,,,,,,,,,,
1a21ab1c498e411929349f40,87.0,56.0,35.0,11.0,11.0,59.0,88.0,NaN,NaN,47.0,...,NaN,25.0,31.0,13.0,18.0,8.0,2.0,2.0,1.0,4.0
4b7812db70c603bb373b92b4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4e8ff234d22dccc37e935193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4f0b05fee4b071c57819891f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502a3170e4b022b4b5336955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5140a733e4b0a19ca1f642b5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5190fefa498e2fe606ae3fee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536fa01911d2228b33b8d208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53f2364b498edb0047f65edd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The Objective of Matrix Factorization is to decompose each user rating into a User-Factor Vector and A Product-Vector. As we have too many NaN’s in our user item rating matrix hence we only solve the ratings which are available i.e. each rating has to be decomposed into 2 vectors.

In [187]:
def matrix_factorization(R,P,Q,steps=100,gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q


Call the matrix_factorization method with all the hyper parameter.

In [188]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=100, gamma=0.001,lamda=0.02)

CPU times: user 21.1 s, sys: 64.7 ms, total: 21.2 s
Wall time: 21.2 s


Now we are done and words are the inout for the user. This is the entry which asks you whihc type of restaurant you are looking for. This data is tokenised after cleaning and words are analysed and the result obtained is matched with the closed value in the matrix and the venue is recommended. Of all only 3 are returned as most appropriate recommendation. 

In [196]:
words = 'I want to eat best food available here' #Write your input here ....

In [ ]:
test_df= pd.DataFrame([words], columns=['Review'])
test_df['Review'] = clean_text(test_df['Review'])
test_vectors = userid_vectorizer.transform(test_df['Review'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Likes'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Likes'],ascending=[0])[:3]

Creating a final_df which will contain all the necessary details of the recommended venue. It has all the columns as regular df but only three entries which are recommended

In [191]:
final_df = pd.DataFrame(columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category',
       'Venue Id', 'Reviews', 'User Id', 'Likes'])
for i in topRecommendations.index.values:
    temp = df.loc[df['Venue Id'] == i]
#     final_df.loc[len(final_df)] = temp
    final_df = final_df.append(temp)

Final Inspection of the data frame

In [192]:
final_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id,Reviews,User Id,Likes
46,Connaught Place,28.631383,77.219792,McDonald's,28.634082,77.219206,Fast Food Restaurant,4dbcce3ef7b1ab37dd690677,patience get table one best mcdonald,53f9a2af498e6618bcace91e,20
59,Connaught Place,28.631383,77.219792,SIDEWOK,28.632906,77.216402,Chinese Restaurant,4eeb625299119cf9ce0e3cd9,coming grp approximate book round table best s...,4f0b05fee4b071c57819891f,4
50,Connaught Place,28.631383,77.219792,Fa Yian,28.632793,77.217188,Chinese Restaurant,4bfd4ffab68d0f476592e857,brilliant value money chinese food like better...,536fa01911d2228b33b8d208,5


Plotting the venues in final_df using venue latitudes and columns...

In [193]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lon, poi in zip(final_df['Venue Latitude'], final_df['Venue Longitude'], final_df['Venue']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

This project concludes here. Hope you understood it. It contains a lot of complicated maths but it works !!!
It can easily be extended to multiple locations. It is based on CP, Delhi due to limited API calls. colde will be 99% same and cre tech used is still valid. Hope to see you there at a recommended location.
Thanks for reviewing.